In [0]:
%sql
    CREATE TABLE IF NOT EXISTS DimCustomerTable3
    (
    surrogateKey BIGINT GENERATED ALWAYS AS IDENTITY,
    CustomerID INT,
    CustomerName STRING,
    City STRING,
    State STRING,
    Country STRING,
    startDate DATE,
    endDate DATE,
    active BOOLEAN
    )
    LOCATION 'dbfs:/mnt/skynet/output/adb_customer_delta2';

In [0]:
%sql

-- Assuming the table `adb_customer_delta2` already exists in the database and has the appropriate schema.
-- Drop the temporary table if it exists
DROP TABLE IF EXISTS tmp_updates;
-- Read the CSV file into a temporary table
CREATE TEMPORARY TABLE tmp_updates
USING csv
OPTIONS (
  path 'dbfs:/mnt/skynetdev/sales_star_schema/customer3.csv',
  header 'true',
  inferSchema 'true'
);


MERGE INTO DimCustomerTable3 AS a
USING
(
  SELECT 
    CustomerID as mergeKey,
    CustomerID,
    CustomerName,
    City,
    State,
    Country
  FROM tmp_updates
  UNION ALL
  SELECT 
    NULL as mergeKey,
    c.CustomerID,
    c.CustomerName,
    c.City,
    c.State,
    c.Country
  FROM tmp_updates as c
  JOIN DimCustomerTable3 as d ON ((c.CustomerID = d.CustomerID) and (c.CustomerName != d.CustomerName) and (d.active=true))
) AS b
ON a.CustomerID = b.mergeKey 
WHEN MATCHED and ((b.CustomerName != a.CustomerName) and (a.active=true)) THEN
update SET a.endDate = current_date() - 1, a.active = 'false'
WHEN NOT MATCHED
THEN 
  INSERT (
    CustomerID,
    CustomerName,
    City,
    State,
    Country,
    startDate,
    endDate,
    active
  )
  VALUES (
    b.CustomerID,
    b.CustomerName,
    b.City,
    b.State,
    b.Country,
    CURRENT_DATE(),  -- Set the startDate to the current date on insert
    '9999-12-31',    -- Set the endDate to null on insert
    'true'
  );


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
3,1,0,2
